<a href="https://colab.research.google.com/github/JavierPalmero23/Bird-Identification-and-Damage-Control-with-Jetson-Nano/blob/main/YOLOv8/BirdDetection/YOLOv8_Bird_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bird Identification with YOLOv8 Model


## **Path**

### **Local Storage**
For Local Storage and Proyect, just make sure to use the complete path from root

In [ ]:

path = ".../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection"


### **Google Drive**
If you want to run another dataset from **Google Drive** you can run:

In [ ]:

#from google.colab import drive
#drive.mount('/content/drive')

#path = "/custom/path/to/datset/and/model/folder"




---



---



## **Training**

### **Repos**

In [ ]:

!pip install ultralytics                                    # if you have problems, check your python verion and the ultralytics instalation
import ultralytics
ultralytics.checks()

import os
import sys

os.environ["PYTHONPATH"]=path

from ultralytics import YOLO


### **Model**
Select the optimal model for you

In [ ]:

# model = YOLO('yolov8n.pt')                                    # New Model From Scratch
# model = YOLO('/content/runs/detect/train/weights/best.pt')    # Load the Best Weight from a Previous Training
model = YOLO(path+'/birdmodel.pt')                              # Load a Pre-Trained Model


### **Train**
Once you selected your model, its time to Run some Training

In [ ]:

results = model.train(data=path+"/data.yaml", epochs=200, pretrained=True, iou=0.5, visualize=True, patience=0)   # Train the Model
results = model.val()                                                                                             # Model Performance Evaluation on the Validation Set


For a longer Training just change the **epoch** value to something higher

### **Save** the Model
Once the Training is done its important to save the model, so you can use it again.

In [ ]:

model.save(path+"/birdmodel.pt")                                # Save the Final Model




---

---



## **Test**

### **Pictures**
The easyest and fastest way to test the Model is with Pictures, so lets get all ready for it

First, import some UTF libraries to avoid problems with the yolo predict command problems

In [ ]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install aspose-words


Now you can Run the Tests.

Just make sure to write the model path. Since the yolo predict is running from the terminal, it will need the full path

In [ ]:

!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://www.womansworld.com/wp-content/uploads/2018/09/birds-and-people-2.jpg'
!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://www.womansworld.com/wp-content/uploads/2018/09/people-and-birds.jpg'
!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://ultralytics.com/images/zidane.jpg'
!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://www.gannett-cdn.com/-mm-/bfca09459d01d9f6efcd53899b98c0f3f6a2e0eb/c=0-138-1817-1160/local/-/media/2018/04/25/USATODAY/usatsports/636601879148177130-canada-goos-attack.jpg'
!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://i.pinimg.com/originals/c3/a2/2f/c3a22f11617e512f880d2125c79dfd70.jpg'
!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://www.mundiario.com/media/mundiario/images/2019/04/03/2019040315042423986.png'
!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://i0.wp.com/www.birdsontheedge.org/wp-content/uploads/2016/12/Photo-by-Romano-da-Costa-1.jpg'
!yolo predict model="/.../Bird-Identification-and-Damage-Control-with-Jetson-Nano/YOLOv8/BirdDetection/birdmodel.pt" source='https://aviancontrolinc.com/wp-content/uploads/2016/09/crow-901602_1280.jpg'


In this case some web pictures are being used, but you can use some local stored ones.

Just replace the address in the source argument

### **Camera**

In [ ]:

!pip install supervision

import cv2
import argparse
from ultralytics import YOLO
import supervision as sv

def parse_arguments()  ->argparse.Namespace:
    parser =argparse.ArgumentParser(description='yolov8 live')
    parser.add_argument("--webcam-resolution", default=[1280,720], nargs=2, type=int)
    args=parser.parse_args()
    return args

def main():
    args=parse_arguments()
    frame_width,frame_height=args.webcam_resolution
    cap=cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT,frame_width)
                                                  # in this point you can change the model.
                                                  #     ex: model=YOLO("yolov8n.pt")
    box_annotator=sv.BoxAnnotator(
        thickness=2,
        text_thickness=2,
        text_scale=1
    )
    while True:
        ret,frame=cap.read()
        result=model(frame)[0]
        detections = sv.Detections.from_yolov8(result)
        labels = [
            f"{model.model.names[class_id]} {confidence:0.2f}"
            for _, confidence,class_id, _
            in detections
        ]
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
        cv2.imshow("YOLOv8",frame)
        print(frame.shape)
        if (cv2.waitKey(30)==27):
            break

if __name__=="__main__":
    main()
